In [1]:
import pickle
from collections import Counter
from datetime import datetime
from pathlib import Path

import joblib
import keras
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    f1_score,
    precision_score,
    recall_score,
)

from utils import *

2024-04-11 00:26:52.038820: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 00:26:52.038871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 00:26:52.039711: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-11 00:26:52.045974: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set up the configuration for the polars library
cfg = pl.Config()
cfg.set_tbl_rows(100)
cfg.set_tbl_width_chars(200)
cfg.set_fmt_str_lengths(200)

pc = ProjectConfig()
BASE_TRAINING_MODEL_DIR = pc.project_root_dir.joinpath("ablation_study").joinpath(
    "model_saves_ab"
)
MODEL_NAMES = [
    "EfficientNetV2M_SINGLE_LAYER",
    "EfficientNetV2M_FOUR_LAYER",
    "EfficientNetV2M_THREE_LAYER",
    "EfficientNetV2M_TWO_LAYER_WIDE_RELU",
    "EfficientNetV2M_TWO_LAYER_WIDE_SWISH",
]
BASE_MODEL_PATHS = []
CLASS_LIST_PATHS = []
BASE_TRAINING_TIME_PATHS = []
FINE_TUNED_TRAINING_TIME_PATHS = []
for model_name in MODEL_NAMES:
    BASE_MODEL_PATHS.append(BASE_TRAINING_MODEL_DIR.joinpath(model_name + ".h5"))
    CLASS_LIST_PATHS.append(
        pc.project_root_dir.joinpath("ablation_study").joinpath(
            f"class_list_{model_name}.lzma"
        )
    )
    BASE_TRAINING_TIME_PATHS.append(
        BASE_TRAINING_MODEL_DIR.joinpath(f"{model_name}_TRAINING_TIME.lzma")
    )

No configuration file found. Using defaults.


In [3]:
TEST_DATA = Path(
    "/data/sampled_test_data_200"
)  # Force Override for Docker predict on GPU
if TEST_DATA.exists() and TEST_DATA.is_dir():
    print("Test data found, loading")
    test_data = tf.keras.utils.image_dataset_from_directory(
        TEST_DATA,
        labels="inferred",
        label_mode="int",
        batch_size=32,
        image_size=(224, 224),
        shuffle=False,
    )
else:
    print("No test data found, where did it go?")

Test data found, loading
Found 5698 files belonging to 69 classes.


2024-04-11 00:26:55.423030: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 00:26:55.428093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 00:26:55.428442: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 00:26:55.429336: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-11 00:26:55.429583: I external/local_xla/xla/stream_executor

In [4]:
results = []

for i, model_path in enumerate(BASE_MODEL_PATHS):
    if model_path.exists():
        print(f"Model {model_path} found, loading")
        tf_model = tf.keras.models.load_model(model_path)
        tf_model.trainable = False
        print(f"Model {model_path} loaded")
    else:
        print(f"Model not found at path: {model_path}")
    if CLASS_LIST_PATHS[i].exists():
        print(f"Class list found at path: {CLASS_LIST_PATHS[i]}, loading")
        class_list = joblib.load(CLASS_LIST_PATHS[i])
        print(f"Class list loaded")

    start_predict = datetime.now()
    predictions = tf_model.predict(test_data)
    joblib.dump(
        predictions,
        BASE_TRAINING_MODEL_DIR.joinpath(f"{MODEL_NAMES[i]}_predictions.lzma"),
    )
    end_predict = datetime.now()
    prediction_time = end_predict - start_predict
    predicted_classes = np.argmax(predictions, axis=-1)
    true_labels = []
    for images, true_label in test_data:
        true_labels.extend(true_label)
    all_predictions = np.array(predicted_classes)
    all_true_labels = np.array(true_labels)
    precision = precision_score(all_true_labels, all_predictions, average="weighted")
    recall = recall_score(all_true_labels, all_predictions, average="weighted")
    accuracy = accuracy_score(all_true_labels, all_predictions)
    f1 = f1_score(all_true_labels, all_predictions, average="weighted")
    base_training_time = joblib.load(BASE_TRAINING_TIME_PATHS[i])
    results.append(
        {
            "model": MODEL_NAMES[i],
            "base_training_time": base_training_time,
            "prediction_time": prediction_time,
            "precision": precision,
            "recall": recall,
            "accuracy": accuracy,
            "f1": f1,
        }
    )
    print(f"Deleting Model {tf_model}")
    del tf_model
    tf.keras.backend.clear_session()


Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_SINGLE_LAYER.h5 found, loading
Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_SINGLE_LAYER.h5 loaded
Class list found at path: /tf/notebooks/gotmeals/ablation_study/class_list_EfficientNetV2M_SINGLE_LAYER.lzma, loading
Class list loaded


2024-04-11 00:27:13.803959: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906


179/179 [==============================] - 23s 97ms/step
Deleting Model <keras.src.engine.functional.Functional object at 0x7f694ed26c90>
Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_FOUR_LAYER.h5 found, loading
Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_FOUR_LAYER.h5 loaded
Class list found at path: /tf/notebooks/gotmeals/ablation_study/class_list_EfficientNetV2M_FOUR_LAYER.lzma, loading
Class list loaded
179/179 [==============================] - 19s 87ms/step
Deleting Model <keras.src.engine.functional.Functional object at 0x7f694f5852d0>
Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_THREE_LAYER.h5 found, loading
Model /tf/notebooks/gotmeals/ablation_study/model_saves_ab/EfficientNetV2M_THREE_LAYER.h5 loaded
Class list found at path: /tf/notebooks/gotmeals/ablation_study/class_list_EfficientNetV2M_THREE_LAYER.lzma, loading
Class list loaded
179/179 [==============================] - 20s 88ms/s

ComputeError: datatype 39m 12s 877216µs cannot be written to csv

In [5]:
results_df = pl.DataFrame(results)
results_df

model,base_training_time,prediction_time,precision,recall,accuracy,f1
str,duration[μs],duration[μs],f64,f64,f64,f64
"""EfficientNetV2M_SINGLE_LAYER""",39m 12s 877216µs,23s 844807µs,0.904781,0.896104,0.896104,0.898334
"""EfficientNetV2M_FOUR_LAYER""",1h 51m 29s 842686µs,19s 895671µs,0.890976,0.875395,0.875395,0.879516
"""EfficientNetV2M_THREE_LAYER""",1h 25m 50s 791347µs,20s 125556µs,0.906318,0.897332,0.897332,0.899683
"""EfficientNetV2M_TWO_LAYER_WIDE_RELU""",1h 1m 54s 946113µs,20s 80483µs,0.911417,0.902948,0.902948,0.905454
"""EfficientNetV2M_TWO_LAYER_WIDE_SWISH""",58m 19s 39618µs,19s 760209µs,0.911452,0.904352,0.904352,0.906264


In [6]:
results_df.write_parquet("alblation_study_results.parquet")